In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import os
import json
import pickle
from transformers import BertTokenizer
import torch
import numpy as np
import re

import warnings
warnings.filterwarnings("ignore")

from tqdm.notebook import tqdm

In [2]:
data_path = "D:/Data/neural-punctuator/szeged_hubert/"
file_path = "D:/Data/neural-punctuator/szeged/szeged.txt"

In [3]:
with open(file_path, 'r', encoding='utf-8') as f:
    text = f.readlines()

In [4]:
from collections import OrderedDict

text = list(OrderedDict.fromkeys(text))
len(text)

80875

In [5]:
from sklearn.utils import shuffle

text = shuffle(text, random_state=0)

text[1]

' Ez jutott az eszébe önkéntelenül.\n'

In [6]:
def clean_text(text):
    text = text.replace('!', '.')
    text = text.replace(':', ',')
    text = text.replace('--', ',')
    text = text.replace('-', ',')
    text = text.replace(';', '.')
    text = text.replace(' ,', ',')
    text = text.replace('♫', '')

    text = re.sub(r'--\s?--', '', text)
    text = re.sub(r'\s+', ' ', text)

    text = text.replace(' ,', ',')
    
    text = re.sub(r',\s?,', ',', text)
    
    return text.strip().lower()

In [31]:
text = [clean_text(t) for t in text]
text[1]

'ez jutott az eszébe önkéntelenül.'

In [32]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("SZTAKI-HLT/hubert-base-cc")
model = AutoModel.from_pretrained("SZTAKI-HLT/hubert-base-cc")

In [33]:
tokenizer.encode(".?,")

[2, 4575, 8308, 3576, 3]

In [54]:
id2target = {-1: 0,
              4575: 1, # .
              8308: 2, # ?
              3576: 3,  # ,
              -2: -1, # will be masked
             }
target2id = {value: key for key, value in id2target.items()}
    
    
def create_target(encoded):
    targets = []
    text = []

    target = -2 # Always mask after [CLS] token

    text.append(encoded[0])
    idx = 1
    while idx < len(encoded):
        word = encoded[idx]
        if word in id2target.keys():
            if word in (4575, 3576) and \
            tokenizer._convert_id_to_token(encoded[idx-1]).lstrip('▁').isnumeric() and \
            tokenizer._convert_id_to_token(encoded[idx+1]).startswith('##') and \
            tokenizer._convert_id_to_token(encoded[idx+1]).strip('#').isnumeric():
                targets.append(-2) # prev number
#                 print(-2)
#                 print(f"{tokenizer._convert_id_to_token(word):15}\t", end="")
                text.append(word)                
                target = -2 # '. or ,'
            else:
                target = word
        else:
            if tokenizer._convert_id_to_token(word).startswith('##'):
                target = -2
            targets.append(target)
#             print(target)
#             print(tokenizer._convert_id_to_token(word), '\t', end="")
            
            target = -1
            text.append(word)
        idx += 1

    targets.append(target)

    targets = [id2target[t] for t in targets]

    return text, targets

In [55]:
train_n = 70_000
valid_n = 8_000

train_text = ' '.join(text[:train_n])
valid_text = ' '.join(text[train_n:train_n+valid_n])
test_text = ' '.join(text[train_n+valid_n:])

len(train_text.split(' ')), len(valid_text.split(' ')), len(test_text.split(' '))

(1080644, 123975, 44242)

In [56]:
train_tokens = tokenizer.encode(train_text)
valid_tokens = tokenizer.encode(valid_text)
test_tokens = tokenizer.encode(test_text)

In [57]:
train_tokens, train_targets = create_target(train_tokens)
valid_tokens, valid_targets = create_target(valid_tokens)
test_tokens, test_targets = create_target(test_tokens)

# For backward campatibility
train_tokens, train_targets = [train_tokens], [train_targets]
valid_tokens, valid_targets = [valid_tokens], [valid_targets]
test_tokens, test_targets = [test_tokens], [test_targets]

In [58]:
for te, ta in zip(train_tokens[0][:100], train_targets[0]):
    print(f"{tokenizer._convert_id_to_token(te):15}\t{ta}")

[CLS]          	-1
az             	0
egyes          	0
_              	0
leszáll        	-1
##ópály        	-1
##ának         	0
például        	0
nem            	0
ez             	0
volt           	0
a              	0
neve           	0
akkoriban      	3
ang            	-1
##li           	-1
##ának         	0
vagy           	0
nagy           	3
brit           	-1
##anni         	-1
##ának         	0
nevezték       	3
bár            	0
l              	-1
##ond          	-1
##ont          	3
ebben          	0
meglehetősen   	0
bizonyos       	0
volt           	0
win            	-1
##st           	-1
##on           	3
mindig         	0
l              	-1
##ond          	-1
##onna         	-1
##k            	0
hívták         	1
ez             	0
jutott         	0
az             	0
eszébe         	0
önként         	-1
##elen         	-1
##ül           	1
könyvtár       	0
megoszt        	-1
##ása          	0
közben         	0
(              	0
a              	0
tulajdonság    	-1
##lap     

In [59]:
with open(data_path + 'train_data.pkl', 'wb') as f:
    pickle.dump((train_tokens, train_targets), f)
with open(data_path + 'valid_data.pkl', 'wb') as f:
    pickle.dump((valid_tokens, valid_targets), f)
with open(data_path + 'test_data.pkl', 'wb') as f:
    pickle.dump((test_tokens, test_targets), f)

In [60]:
sum((np.array(vt) != -1).sum() for vt in train_targets), \
sum((np.array(vt) != -1).sum() for vt in valid_targets), \
sum((np.array(vt) != -1).sum() for vt in test_targets)

(1153158, 132224, 46992)

In [61]:
tokenizer.decode(train_tokens[0][:30])

'[CLS] az egyes _ leszállópályának például nem ez volt a neve akkoriban angliának vagy nagy britanniának nevezték bár londont ebben meglehetősen bizonyos'

In [62]:
word_idx = 0
s = []
for te, ta in zip(test_tokens[0][:], test_targets[0]):
#     print(tokenizer._convert_id_to_token(te), ta)
    if ta != -1:
        s.append(te)
        decoded = tokenizer.decode(s)
        print(f"{word_idx:3}. {decoded:20}\t{ta}\t{[tokenizer._convert_id_to_token(ss) for ss in s]}")
        word_idx += 1
        s = []
    else:
        s.append(te)

  0. [CLS] hála          	0	['[CLS]', 'hála']
  1. az                  	0	['az']
  2. úrnak               	3	['úrnak']
  3. nagyon              	0	['nagyon']
  4. jó                  	0	['jó']
  5. képességeim         	0	['képessége', '##im']
  6. vannak              	3	['vannak']
  7. és                  	0	['és']
  8. nagyon              	0	['nagyon']
  9. szívesen            	0	['szívesen']
 10. tanítanék           	0	['tanít', '##an', '##ék']
 11. egyetemeken         	3	['egyetemek', '##en']
 12. főiskolákon         	0	['főiskol', '##ákon']
 13. is                  	0	['is']
 14. teológusokat        	3	['te', '##ológusok', '##at']
 15. filozófusokat       	1	['filozófus', '##okat']
 16. ha                  	0	['ha']
 17. az                  	0	['az']
 18. ember               	0	['ember']
 19. mindig              	0	['mindig']
 20. úgy                 	0	['úgy']
 21. gondolkozna         	3	['gondolkoz', '##na']
 22. mint                	0	['mint']
 23. most                	0	['most'

958. szervezetben        	0	['szervezetben']
959. dolgozó             	0	['dolgozó']
960. emberek             	0	['emberek']
961. kiválasztása        	0	['kiválasztása']
962. és                  	0	['és']
963. megbízhatóságuk     	0	['megbízhatóság', '##uk']
964. ellenőrzése         	0	['ellenőrzése']
965. rendkívül           	0	['rendkívül']
966. kényes              	0	['kényes']
967. kérdés              	1	['kérdés']
968. június              	0	['június']
969. 7                   	3	['7']
970. én                  	0	['én']
971. a                   	0	['a']
972. magyar              	0	['magyar']
973. _                   	0	['_']
974. kupa                	0	['kup', '##a']
975. fináléjával         	0	['finálé', '##jával']
976. ért                 	0	['ért']
977. volna               	0	['volna']
978. véget               	0	['véget']
979. az                  	0	['az']
980. idény               	3	['idény']
981. de                  	0	['de']
982. a                   	0	['a']
983. luxemburgi

1870. életet              	0	['életet']
1871. élők                	0	['élők']
1872. sem                 	0	['sem']
1873. képesek             	0	['képesek']
1874. kisebb              	0	['kisebb']
1875. változtatásra       	0	['változtatás', '##ra']
1876. egyik               	0	['egyik']
1877. napról              	0	['napról']
1878. a                   	0	['a']
1879. másikra             	3	['másikra']
1880. egyedül             	1	['egyedül']
1881. a                   	0	['a']
1882. terjesztés          	0	['terjeszt', '##és']
1883. magában             	0	['magában']
1884. foglalja            	0	['foglalja']
1885. különösen           	0	['különösen']
1886. a                   	0	['a']
1887. műpéldány           	0	['mű', '##péld', '##ány']
1888. tulajdonjogának     	0	['tulajdonjog', '##ának']
1889. átruházását         	0	['átruház', '##ását']
1890. és                  	0	['és']
1891. a                   	0	['a']
1892. műpéldány           	0	['mű', '##péld', '##ány']
1893. bérbeadását     

2767. baj                 	0	['baj']
2768. mégis               	0	['mégis']
2769. nagy                	3	['nagy']
2770. egy                 	0	['egy']
2771. belső               	0	['belső']
2772. vizsgálat           	3	['vizsgálat']
2773. amelyet             	0	['amelyet']
2774. az                  	0	['az']
2775. özvegy              	3	['özvegy']
2776. gergely             	0	['ger', '##gel', '##y']
2777. _                   	0	['_']
2778. irén                	0	['ir', '##én']
2779. kért                	0	['kért']
2780. a                   	0	['a']
2781. p                   	1	['p']
2782. _                   	0	['_']
2783. m                   	1	['m']
2784. _                   	0	['_']
2785. s                   	1	['s']
2786. _                   	0	['_']
2787. kft                 	3	['kft']
2788. től                 	3	['től']
2789. azt                 	0	['azt']
2790. állapította         	0	['állapította']
2791. meg                 	3	['meg']
2792. hogy                	0	['hogy']
2793

3788. rozzant             	0	['roz', '##zan', '##t']
3789. kalapját            	3	['kalap', '##ját']
3790. és                  	0	['és']
3791. belépett            	0	['belép', '##ett']
3792. a                   	0	['a']
3793. füstös              	0	['füst', '##ös']
3794. helyiségbe          	1	['helyiség', '##be']
3795. mi                  	0	['mi']
3796. az                  	0	['az']
3797. ott                 	0	['ott']
3798. előtted             	3	['előtte', '##d']
3799. öregfiú             	3	['öreg', '##fiú']
3800. de                  	0	['de']
3801. jó                  	0	['jó']
3802. lehetett            	1	['lehetett']
3803. miután              	0	['miután']
3804. mindenki            	0	['mindenki']
3805. jóllakott           	3	['jól', '##lak', '##ott']
3806. megkérdeztük        	0	['megkérd', '##eztük']
3807. mikor               	0	['mikor']
3808. kell                	0	['kell']
3809. visszajönni         	0	['visszaj', '##ön', '##ni']
3810. és                  	0	['és']
3811. me

...

42342. csak                	0	['csak']
42343. vernét              	0	['ver', '##né', '##t']
42344. gúnyolta            	3	['gúny', '##olta']
42345. de                  	0	['de']
42346. a                   	0	['a']
42347. városi              	0	['városi']
42348. közlekedésről       	0	['közlekedés', '##ről']
42349. is                  	0	['is']
42350. máig                	0	['máig']
42351. érvényes            	0	['érvényes']
42352. véleményt           	0	['véleményt']
42353. mondott             	3	['mondott']
42354. amikor              	0	['amikor']
42355. 1912                	3	['191', '##2']
42356. ben                 	0	['ben']
42357. megírta             	0	['megírt', '##a']
42358. a                   	0	['a']
42359. _                   	0	['_']
42360. csömöri             	0	['cs', '##ömör', '##i']
42361. _                   	0	['_']
42362. úttól               	0	['útt', '##ól']
42363. _                   	0	['_']
42364. a                   	0	['a']
42365. _                   	0	['_'

43196. neki                	3	['neki']
43197. mert                	0	['mert']
43198. nem                 	0	['nem']
43199. sikerült            	0	['sikerült']
43200. tisztázni           	3	['tisztázni']
43201. hogy                	0	['hogy']
43202. milyen              	0	['milyen']
43203. arányban            	0	['arányban']
43204. támogatja           	0	['támogatja']
43205. a                   	0	['a']
43206. kormány             	0	['kormány']
43207. a                   	0	['a']
43208. dolgot              	1	['dolgot']
43209. először             	0	['először']
43210. kígyót              	0	['kígy', '##ót']
43211. táncoltatott        	0	['tánc', '##olta', '##to', '##tt']
43212. sípszó              	0	['síp', '##szó']
43213. mellett             	3	['mellett']
43214. azután              	0	['azután']
43215. teleszúrta          	0	['tele', '##sz', '##úrt', '##a']
43216. a                   	0	['a']
43217. testét              	0	['testét']
43218. tűkkel              	1	['tű', '##k', '##kel'

44078. mosolyog            	3	['mosolyog']
44079. és                  	0	['és']
44080. mintegy             	0	['mintegy']
44081. oda                 	0	['oda']
44082. sem                 	0	['sem']
44083. nézve               	3	['nézve']
44084. könnyedén           	0	['könnyedén']
44085. tölti               	0	['tölti']
44086. a                   	0	['a']
44087. teát                	3	['teát']
44088. de                  	0	['de']
44089. közben              	3	['közben']
44090. hogy                	3	['hogy']
44091. hogy                	0	['hogy']
44092. nem                 	3	['nem']
44093. a                   	0	['a']
44094. forró               	0	['forró']
44095. vízsugár            	0	['víz', '##sugár']
44096. egy                 	0	['egy']
44097. idős                	0	['idős']
44098. úr                  	0	['úr']
44099. kopasz              	0	['kop', '##asz']
44100. fejére              	0	['fej', '##ére']
44101. ömlik               	1	['öm', '##lik']
44102. mihály              	0	

44864. sem                 	0	['sem']
44865. lehet               	0	['lehet']
44866. eltérni             	1	['eltér', '##ni']
44867. az                  	0	['az']
44868. a                   	0	['a']
44869. gondolat            	0	['gondolat']
44870. nélküli             	0	['nélküli']
44871. gyengédség          	0	['gyengéd', '##ség']
44872. azonban             	3	['azonban']
44873. amely               	0	['amely']
44874. a                   	0	['a']
44875. mogyoróbokrok       	0	['mogyor', '##ób', '##ok', '##rok']
44876. mellett             	0	['mellett']
44877. fogta               	0	['fogta']
44878. el                  	3	['el']
44879. most                	0	['most']
44880. nem                 	0	['nem']
44881. lepte               	0	['lep', '##te']
44882. meg                 	0	['meg']
44883. újból               	1	['újból']
44884. (                   	0	['(']
44885. lásd                	0	['lásd']
44886. a                   	0	['a']
44887. 275                 	1	['27', '##5']
44888.

45761. megismerhetőségét   	0	['megismer', '##hetőség', '##ét']
45762. érintő              	0	['érintő']
45763. rendelkezést        	0	['rendelkezés', '##t']
45764. állapított          	0	['állap', '##ított']
45765. meg                 	3	['meg']
45766. amikor              	0	['amikor']
45767. a                   	0	['a']
45768. 2                   	1	['2']
45769. par                 	1	['par']
45770. (                   	0	['(']
45771. 1                   	0	['1']
45772. )                   	0	[')']
45773. bekezdésében        	0	['bekezdésében']
45774. megjelölt           	0	['megjelölt']
45775. szervek             	0	['szervek']
45776. és                  	0	['és']
45777. szervezetek         	0	['szervezetek']
45778. számára             	0	['számára']
45779. a                   	0	['a']
45780. művészeti           	0	['művészeti']
45781. alkotások           	0	['alkotások']
45782. beszerzését         	3	['beszerzés', '##ét']
45783. felhasználását      	0	['felhasználását']
45784. és  